- Average Annual Household Transportation Costs: Out-of-pocket expenses including tolls, parking, and transit fares

    - These costs do not include other vehicle operating and purchase costs.

    - Costs are annualized using factor of 300, as used in the 2018 RTP.


In [1]:
import pandas as pd
import polars as pl
import toml
from pathlib import Path
from sqlalchemy import create_engine

config = toml.load(Path.cwd() / '../../../../configuration/input_configuration.toml')
summary_config = toml.load(Path.cwd() / '../../../../configuration/summary_configuration.toml')

pd.set_option('display.float_format', '${:0,.0f}'.format)

In [ ]:
# Relative path between notebooks and goruped output directories
output_path = Path(summary_config['sc_run_path']) / summary_config["output_folder"]
survey_path = Path(summary_config['sc_run_path']) / summary_config["survey_folder"]

# Write out CSVs with a numeric value prefix to help organize outputs 
incr = 1

In [10]:
# person data
person = pd.read_csv(output_path / 'agg/dash/person_cost.csv')
# household data
hh = pd.read_csv(output_path / 'agg/dash/hh_geog.csv')


# list of equity geographies
equity_geogs = summary_config['equity_geogs']
not_equity_geogs = ["NOT in " + item for item in equity_geogs]

In [4]:
annual_factor = 300

df_person = person.copy()
df_person['is_rgc'] = "In RGC"
df_person.loc[df_person['hh_rgc'] != 'Not in RGC', 'is_rgc'] = "Not in RGC"
# included modes
# TODO: ask what this means
df_person = df_person[(df_person['mode'].isin(['SOV','HOV2','HOV3+']) & (df_person['dorp'] == 1)) 
                      | (~df_person['mode'].isin(['SOV','HOV2','HOV3+']))]

df_hh = hh.copy()
df_hh['is_rgc'] = "In RGC"
df_hh.loc[df_hh['hh_rgc'] != 'Not in RGC', 'is_rgc'] = "Not in RGC"

In [5]:
def annual_costs(geog):
    df = df_person.groupby(geog, as_index=False)['travcost_wt'].sum().set_index(geog)
    df['hhexpfac'] = df_hh.groupby(geog)['hhexpfac'].sum()

    df.loc['Region'] = [df_person['travcost_wt'].sum(), df_hh['hhexpfac'].sum()]
    df['Average Annual Costs'] = (df['travcost_wt'] / df['hhexpfac']) * annual_factor

    return df[['Average Annual Costs']]

In [6]:
annual_costs('hh_county')

,Average Annual Costs
hh_county,
King,"$2,180"
Kitsap,"$2,463"
Outside Region,"$2,608"
Pierce,"$2,540"
Snohomish,"$2,785"
Region,"$2,382"


In [7]:
annual_costs('is_rgc')

,Average Annual Costs
is_rgc,
In RGC,"$2,540"
Not in RGC,$889
Region,"$2,382"


In [8]:
annual_costs('hh_rgc')

,Average Annual Costs
hh_rgc,
Auburn,"$1,703"
Bellevue,$931
Bothell Canyon Park,"$2,373"
Bremerton,"$1,123"
Burien,"$1,674"
Everett,"$1,081"
Federal Way,"$1,864"
Greater Downtown Kirkland,"$1,564"
Kent,"$1,587"


In [9]:
annual_costs('hh_rg_proposed')

,Average Annual Costs
hh_rg_proposed,
CitiesTowns,"$3,150"
Core,"$2,358"
HCT,"$2,552"
Metro,"$1,565"
UU,"$2,954"
Region,"$2,382"


In [19]:
# get total intersections by equity geography
df1 = df_person[equity_geogs].apply(lambda x: x * df_person['travcost_wt']).sum().reset_index()
df1.columns = ['Equity Group', 'travcost_wt']
df1.set_index('Equity Group', inplace=True)
df1['hhexpfac'] = df_hh[equity_geogs].apply(lambda x: x * df_hh['hhexpfac']).sum()

df1['Inside Equity Geography'] = (df1['travcost_wt'] / df1['hhexpfac']) * annual_factor

# outside equity geographies
df2 = df_person[equity_geogs].apply(lambda x: (1-x) * df_person['travcost_wt']).sum().reset_index()
df2.columns = ['Equity Group', 'travcost_wt']
df2.set_index('Equity Group', inplace=True)
df2['hhexpfac'] = df_hh[equity_geogs].apply(lambda x: (1-x) * df_hh['hhexpfac']).sum()

# add outside equity geographies costs
df1['Outside Equity Geography'] = (df2['travcost_wt'] / df2['hhexpfac']) * annual_factor

df1[['Inside Equity Geography','Outside Equity Geography']]

,Inside Equity Geography,Outside Equity Geography
Equity Group,,
hh_disability_reg,"$2,286","$2,462"
hh_elderly_reg,"$2,479","$2,294"
hh_english_reg,"$2,353","$2,399"
hh_poverty_reg,"$2,139","$2,544"
hh_racial_reg,"$2,196","$2,538"
hh_youth_reg,"$2,828","$2,015"
hh_poverty_50,"$1,594","$2,399"
hh_racial_50,"$2,188","$2,469"
